# Задание 4
Пан Анатолий Эдуардович

группа 932209

Вариант 2. Регулярные языки.

Разработать программу, которая по данному регулярному выражению строит конечный
автомат, распознающий соответствующий регулярный язык, и осуществляет проверку
строк на принадлежность этому языку. Для упрощения можно ввести ограничения на
вложенность скобок.

In [ ]:
class RegexMatcher(object):
    def isMatch(self, input_str: str, pattern: str):
        """
        :type input_str: str
        :type pattern: str
        :rtype: bool
        """

        # Цикл для построения NDFSA (недетерминированный конечный автомат с ε-переходами)
        start_state = NDFSAState()
        current_state = start_state
        i = 0
        while i < len(pattern):
            char_pattern = pattern[i]
            next_state = NDFSAState()

            # Если kleene_star, добавляем переход (current_state, char_pattern) -> current_state,
            # и добавляем next_state в epsilons текущего состояния. Это означает, что
            # мы можем потреблять любое количество символов char_pattern и оставаться в текущем состоянии,
            # либо перейти к следующему состоянию, не используя входные данные
            # (что является определением *)
            # В противном случае добавляем переход от (current_state, char_pattern) -> next_state
            kleene_star = i < len(pattern) - 1 and pattern[i + 1] == '*'
            if char_pattern == '.':
                for j in range(26):
                    char = chr(ord('a') + j)
                    current_state.transitions[char] = current_state if kleene_star else next_state
            else:
                current_state.transitions[char_pattern] = current_state if kleene_star else next_state
            if kleene_star:
                i += 1
                current_state.epsilons.add(next_state)
            current_state = next_state
            i += 1

        # Финальное состояние по определению является конечным состоянием
        accept_state = current_state

        # Цикл для симуляции NDFSA на строке input_str
        # Согласно определению NDFSA, если любая последовательность переходов приводит к
        # конечному состоянию, то строка принимается. Для этой симуляции мы
        # сохраняем множество возможных состояний (current_states) после того, как
        # n символов будут использованы из input_str. Мы начинаем с начального состояния и его эпсилон-замыкания
        # для n = 0. Для любого данного n это множество состояний будет объединением
        # эпсилон-замыканий всех состояний, в которые можно перейти из предыдущего множества
        # состояний, используя input_str[n]

        # Мы принимаем строку только в том случае, если конечное состояние присутствует в
        # наших конечных состояниях
        current_states = set([start_state]) | start_state.epsilon_closure()
        for c in input_str:
            next_states = set()
            for state in current_states:
                if c in state.transitions:
                    next_state = state.transitions[c]
                    next_states |= set([next_state]) | next_state.epsilon_closure()
            current_states = next_states
        return accept_state in current_states

class NDFSAState(object):
    def __init__(self):
        """
        Эпсилон обычно является символом для null символа
        Это означает, что для перехожа не используются никакие входные данные
        epsilons - это множество состояний, в которые мы можем перейти без необходимости входных данных
        """
        self.epsilons = set()
        self.transitions = {}  # char_input -> state

    # Этот метод вычисляет множество всех состояний, в которые может перейти это состояние
    # без необходимости ввода. Для этого нам нужно рекурсивно посетить все узлы
    # в epsilons и накопить все состояния
    def epsilon_closure(self):
        result = set([self])
        for state in self.epsilons:
            result |= state.epsilon_closure()
        return result

In [ ]:
solution = RegexMatcher()

pattern = "ab*c"
test_strings = ["ab", "abbbc", "ac", "abc"]
for string in test_strings:
    result = solution.isMatch(string, pattern)
    print(f"For regex pattern: {pattern} and string: {string} result is {result}")  # Should print True


For regex pattern: ab*c and string: ab result is False
For regex pattern: ab*c and string: abbbc result is True
For regex pattern: ab*c and string: ac result is True
For regex pattern: ab*c and string: abc result is True
